<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 作業用ディレクトリへ移動
# システムコマンドは基本「！」をつけるが、ディレクトリ移動は「％」らしい
!mkdir chap6
%cd ./chap6


/content/chap6


In [2]:
# いつものライブラリに加えて、ファインチューニングと性能評価を行うPyTorch Lightningも使用
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# またNo module named ‘torchtext.legacy’エラーが出たため
# https://masaki-note.com/2022/05/27/torchtext_legacy_error/
# に従い以下を実行
# !pip install --upgrade torchtext==0.9.1

# いろいろなライブラリたち
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification


# 日本語事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 24.9 MB/s 
     |████████████████████████████████| 486 kB 57.2 MB/s 
     |████████████████████████████████| 13.4 MB 40.2 MB/s 
     |████████████████████████████████| 582 kB 65.0 MB/s 
     |████████████████████████████████| 6.6 MB 45.4 MB/s 
     |████████████████████████████████| 880 kB 12.4 MB/s 
     |████████████████████████████████| 120 kB 24.9 MB/s 
     |████████████████████████████████| 419 kB 74.6 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=241bbec780125c0fc98b9bb6900c7fbc503e423697dbcc3322e36247364d1a64
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=79103b22fe59588f48e5136a0a2f0d43152bbb7c3982c2819b9f0cd46aa125cf
  Stored in dire

In [3]:
import pytorch_lightning as pl

In [4]:
# ６章のお題は文章分類
# まずはネガポジ分類からやってみよう
# 入力された文章がポジティブならラベル１、ネガティブならラベル０を返すぞ！

# まずはトークナイザを定義
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# 分類器を定義
# ねがポジ日分類なのでnum_labelはs2
# GPUに乗せておく
bert_sc = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)
bert_sc = bert_sc.cuda()



Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [5]:
''' 
BertForSequenceClassificationには学習と推論の２フェーズがある

推論：符号化した文章を入力して、各カテゴリに対する分類スコアを出力
  分類スコアは２次元配列
  入力のi文目のラベルjに対する分類スコアはscore[i, j]

学習時：符号化した文章とラベル（カテゴリ）を入力として、損失の値を出力


予測性能評価には「精度」という指標を用いる
精度　＝　予測が正しかったデータの数/総データ数 (= 再現率？)

BertForSequenceClassificationはBertModelの最終層の出力のうち、[CLS]に対応する出力に対して、
線形変換、tanh関数、線形変換が適用されて分類スコア計算されている
'''


' \nBertForSequenceClassificationには学習と推論の２フェーズがある\n\n推論：符号化した文章を入力して、各カテゴリに対する分類スコアを出力\n  分類スコアは２次元配列\n  入力のi文目のラベルjに対する分類スコアはscore[i, j]\n\n学習時：符号化した文章とラベル（カテゴリ）を入力として、損失の値を出力\n\n\n予測性能評価には「精度」という指標を用いる\n精度\u3000＝\u3000予測が正しかったデータの数/総データ数 (= 再現率？)\n\n\n'

In [22]:
# じゃあ実際ネガポジ分類してみようぜ

# 学習用テキスト（レビュー文）
text_list = [
    'この映画は面白かった。',
    'この映画の最後にはガッカリさせられた。',
    'この映画を見て幸せな気持ちになった。'
]

# 1 がポジティブ
label_list = [1, 0, 1]

# データの符号化
encoding = tokenizer(
    text_list,
    padding = 'longest',
    return_tensors='pt'
)


# GPUにのっける
# ちなみに、cuda()はpytorchが提供しているメソッドなので、torch.tensorなどのデータ型にしか使えない
encoding = { k : v.cuda() for k, v in encoding.items()}
labels = torch.tensor(label_list).cuda()
print(type(encoding))
print(type({*encoding}))
print(type({**encoding}))


# 推論フェーズ
# forward は分類スコア（など）を返す。
# 分類スコアは「logits」とタグづけされている。
# 詳しくはhttps://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification.forward
with torch.no_grad():
  output = bert_sc.forward(**encoding)

scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1) # スコア最大のインデックス = カテゴリー番号的なやつ
num_correct = (labels_predicted==labels).sum().item() # 正解数
print(type(labels_predicted))
print(labels_predicted==labels)
print((labels_predicted==labels).sum())

# 精度
accuracy = num_correct / labels.size(0) # 精度
print(labels.size(0))
print(labels.size())


print("# scoresのサイズ：")
print(scores.size())

print("# predicted labels:")
print(labels_predicted)

print("# accuracy:")
print(accuracy)



<class 'dict'>
<class 'set'>
<class 'dict'>
<class 'torch.Tensor'>
tensor([False,  True, False], device='cuda:0')
tensor(1, device='cuda:0')
3
torch.Size([3])
# scoresのサイズ：
torch.Size([3, 2])
# predicted labels:
tensor([0, 0, 0], device='cuda:0')
# accuracy:
0.3333333333333333


In [ ]:
'''
BERTのファインチューニング
符号化した文章を入力して、出力とラベルの間の損失を計算する

損失関数：スコアをSoftmax関数で各ラベルの予測確率に変換し、それと実際のラベルとの間の交差エントロピー
          こいつを最小化したい

損失さえ計算しちゃえば、パラメータ更新作業はPyTorch Lightningが自動でやってくれる

'''

In [28]:
# さて、損失関数を計算しよう


# 符号化
encoding = tokenizer(
    text_list,
    padding = 'longest',
    return_tensors = 'pt'
)
print(encoding)
print(encoding.items())

# 入力にラベルを追加
# BertForSequenceClassificationは入力にラベル(labels)が含まれる場合に損失を出力
encoding ['labels'] = torch.tensor(label_list)

# GPU送りにする
encoding = { k : v.cuda() for k, v in encoding.items() }


# ロス計算
output = bert_sc(**encoding)
loss = output.loss

print("# loss :")
print(loss)

{'input_ids': tensor([[    2,    70,   450,     9,  9727,   187,    10,     8,     3,     0,
             0,     0,     0,     0,     0,     0],
        [    2,    70,   450,     5,  1133,     7,     9,   444,  3259, 28479,
            26,   191,    84,    10,     8,     3],
        [    2,    70,   450,    11,   212,    16, 13215,    18,  8415,     7,
            58,    10,     8,     3,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}
dict_items([('input_ids', tensor([[    2,    70,   450,     9,  9727,   187,    10,     8,     3,     0,
             0,     0,     0,     0,     0,     0],
        [    2,    70,   450,     5,  1133,   